In [1]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from pandas.plotting import register_matplotlib_converters

import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import xarray as xr
import atlite
from shapely.geometry import Point
from shapely.ops import unary_union
import numpy as np
import rasterio
import shapely

# from dask.distributed import Client, LocalCluster

# nprocesses = 20
# cluster = LocalCluster(n_workers=nprocesses, threads_per_worker=2)
# client = Client(cluster, asynchronous=True)

# resource={}
# resource["dask_kwargs"] = {"scheduler": client}

import PySAM.Pvwattsv8 as pv
from concurrent.futures import ProcessPoolExecutor

In [20]:
re.split(',\s*', '2018,       2019')

['2018', '2019']

In [15]:
years="2021,2022"

# Convert years to string if it's not already
if not isinstance(years, str):
    years = str(years)

# Remove square brackets
years_string = years.strip('[]')

# Split by comma (with or without space) and convert to integer
years_list = list(map(int, re.split(',\s*', years_string)))

In [16]:
years_list

[2021, 2022]

In [3]:
rsa_supply = gpd.read_file('../data/bundle/rsa_supply_regions.gpkg', layer='1-supply')

In [4]:
cutout = atlite.Cutout(
    path="../cutouts/RSA-2017_20-era5.nc",
    chunks={"time": 100},
)
cutout.data = cutout.data.sel(time=slice('2019-01-01', '2019-12-31'))

# pv_sat = cutout.pv(    
#     panel="CSi",
#     shapes=cutout.grid,
#     tracking="horizontal",
#     per_unit=True, 
#     orientation={"slope": 0.0, "azimuth": 0},
#     **resource,
# )

# pv_ft = cutout.pv(    
#     panel="CSi",
#     shapes=cutout.grid,
#     tracking=None,
#     per_unit=True, 
#     orientation="latitude_optimal", # or {"slope": 0.0, "azimuth": 0},
#     **resource,
# )

# pv_ft.sel(dim_0=100).plot(color='r')
# pv_sat.sel(dim_0=100).plot()
cutout

<Cutout "RSA-2017_20-era5">
 x = 16.20 ⟷ 34.80, dx = 0.30
 y = -34.80 ⟷ -21.00, dy = 0.30
 time = 2019-01-01 ⟷ 2019-12-31, dt = H
 module = era5
 prepared_features = ['height', 'wind', 'influx', 'temperature', 'runoff']

In [10]:
cutout.grid()

TypeError: 'GeoDataFrame' object is not callable

In [5]:
for i in range(0,200):
    extract_solar(cutout, i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86


KeyboardInterrupt: 

In [1]:
def extract_solar(cutout, cell):
    pv_inputs ={
		"system_capacity" : 1*1.2,
		"module_type" : 1,
		"dc_ac_ratio" : 1.2,
		"bifaciality" : 0,
		"array_type" : 0,
		"azimuth" : 0,
		"gcr" : 0.4,
		"soiling" : [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
		"losses" : 11.42,
		"en_snowloss" : 0,
		"inv_eff" : 96,
		"batt_simple_enable" : 0,
        'tilt': 30,
	}
    
    lat = cutout.grid.loc[cell,'y']
    long = cutout.grid.loc[cell,'x']
    dn = cutout.data.sel(x=long,y=lat)['influx_direct'].values.tolist()
    dh = cutout.data.sel(x=long,y=lat)['influx_diffuse'].values.tolist()
    tdry = (cutout.data.sel(x=long,y=lat)['temperature'].values-273.15).tolist()
    elev = (cutout.data.sel(x=long,y=lat)['height'].values).tolist()
    
    model = pv.new()#default('PVWattsSingleOwner')
    for k, v in pv_inputs.items():
        if k != 'number_inputs':
            setattr(model.SystemDesign, k, v)
    model.AdjustmentFactors.constant = 0

    wspd = [0]*8760
    dates = pd.date_range('2019-01-01 00:00', '2019-12-31 23:00', freq='H')
    year = list(dates.year)
    month = list(dates.month)
    day = list(dates.day)
    hour = list(dates.hour)
    minute = list(dates.minute)

    model.SolarResource.assign(
        {
            'solar_resource_data':{
                'dn':dn,
                'df':dh,
                'tdry':tdry,
                'wspd':wspd,
                'lat':lat,
                'lon':long,
                'tz':2,
                'elev':elev,
                'year':year,
                'month':month,
                'day':day,
                'hour':hour,
                'minute':minute,
                'albedo':0.2,
                "use_wf_albedo":0,
                }
        }
    )
    model.execute()
    print(cell)
    return model.Outputs.capacity_factor

In [20]:
from dask import delayed, compute

# Assuming your function extract_solar and cutout object are already defined

# Create delayed objects for each function call
tasks = [delayed(extract_solar)(cutout, cell) for cell in cutout.grid.index[0:200]]

# Compute the results in parallel
results = compute(*tasks)

print(results)


80
193
107
59110

62
64
76
143
111
3
13
105
74
43
113
54
172
141
36
46179
77

1
5
32
132
82
53
161
90
48
58
89
187
109
8
121
52
61
112
66
197
135
71
180
11
45
136
183
12010
6
127
185

119
499
138
176
92
158

20
73
166
23
178
97
50
145
137
53
88
150
110
81
142
77
181
131
3323
71
107
104187

65
39
193

70
59
13064

124
54
160
135
159
172
37
179
92
78
147
186
57
158
18140
63
80
15525

175
152
24
35100


2
67
189
9190
29
82
84
99
185
12
191
176
85
184
0

79170

93
149
173
15196

30
52
196
109
146
5
174
126
86
105
32
8
68
83
21
123
114
28
183
44
17
7153
13666

20
15
169
42
118
192
34
117
10
74

106163
22

11638
167
61
157
98
4156
73
133194


72
132
3
182
102
95
69
60
121
148
19188

55
111
27
138

49
14
43
166
90
75
51
141
164
139
62
41
113
165
129
40
115
48
58
31
195
13
162
143
26
94
6
134
16
119
177
197
125
198108
180
103
171
120
161
154
16847
91

127
89
76
199
112
101
11
56

46
1
45
128122

87
144
36
(12.945120582854392, 13.020559039726301, 13.079324568644106, 13.134494876639454, 13.16361

In [ ]:
# nprocesses = 20

# with ProcessPoolExecutor(max_workers=nprocesses) as executor:
#     futures = {
#         cell: executor.submit(
#             extract_solar,
#             cutout = cutout,
#             cell = cell,
#             )
#         for cell in cutout.grid.index[0:100]
#     }

KeyboardInterrupt: 